In [ ]:
from datetime import date

today = date.today()

In [ ]:
from erddapy import ERDDAP


server = "https://gliders.ioos.us/erddap"
protocol = "tabledap"


e = ERDDAP(server=server, protocol=protocol)

In [ ]:
import requests


from urllib.parse import quote
from erddapy.url_handling import urlopen
import pandas as pd


url_dset = (
    f"{e.server}"
    "/tabledap/allDatasets.csv?"
    "datasetID,minLongitude,minLatitude,minTime,maxTime,institution"
    f"&maxTime>=now-1days"
)

url_dataset = quote(url_dset, safe=":/?&= ")

df = pd.read_csv(urlopen(url_dataset), skiprows=[1])

cols = ["minTime", "maxTime"]
df[cols] = df[cols].apply(pd.to_datetime)

In [ ]:
import folium


def make_marker(df):
    link = (
        f"{server}/"
        f"{protocol}/"
        f"{df['datasetID']}.html"
    )
    popup = folium.Popup(
        html=f"""{df['institution']} glider <a href="{link}">{df['datasetID']}</a""",
    )
    location = [df["maxLatitude"], df["maxLongitude"]]
    
    icon = folium.Icon(color="orange", icon="glyphicon glyphicon-plane")
    return folium.Marker(location=location, popup=popup, icon=icon)


tiles = "https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}"
attr = "Tiles &copy; Esri &mdash; Sources: GEBCO, NOAA, CHS, OSU, UNH, CSUMB, National Geographic, DeLorme, NAVTEQ, and Esri"

m = folium.Map(tiles=tiles, attr=attr, png_enabled=True, zoom_control=False)

for k, row in df.iterrows():
    make_marker(row).add_to(m)

m.fit_bounds(m.get_bounds())

m

In [ ]:
import pathlib


pathlib.Path("images/").mkdir(parents=True, exist_ok=True)


with open(f"images/{today:%Y-%m-%d}-map.png", "wb") as f:
    f.write(m._to_png(delay=3))

In [ ]:
from urllib.parse import quote_plus
from erddapy.erddapy import parse_dates


def glider_track_png(df):
    dataset_id = df["datasetID"]
    min_time = parse_dates(df["minTime"])
    max_time = parse_dates(df["maxTime"])

    variable = "salinity,temperature"
    graph = (
        f"{variable}"
        f"&time>={min_time}"
        f"&time<={max_time}"
        f"&distinct()"
        f"&.draw=markers&.marker=5|5&.color=0x000000&.colorBar=|||||&.bgColor=0xffccccff"
    )
    img = f"{server}/{protocol}/{dataset_id}.smallPng?{quote_plus(graph)}"
    return f'<img src="{img}" alt="glider">'

In [ ]:
from erddapy.url_handling import urlopen
from shapely.geometry import LineString


def request_track(dataset_id):
    df = None
    protocol = "tabledap"
    variables = ["time", "longitude", "latitude"]
    # we will display only the glider's last teo week to reduce the data.
    constraints = {
        "time>=": "now-14days",
    }
    url = e.get_download_url(
        protocol=protocol,
        dataset_id=dataset_id,
        variables=variables,
        constraints=constraints,
        response="csvp",
        distinct=True,
    )
    try:
        df = pd.read_csv(urlopen(url))
        df.name = dataset_id
    except requests.exceptions.HTTPError:
        print(f"Failed to fetch glider {dataset_id}.")
        pass
    return df


def plot_track(df, popup, color="orange"):
    locations = list(zip(df["latitude (degrees_north)"].tolist(), df["longitude (degrees_east)"]))

    folium.PolyLine(
        locations=locations, color=color, weight=8, opacity=0.75, tooltip=df.name, popup=popup,
    ).add_to(m)

    kw = {"fill": True, "radius": 7, "stroke": False}
    folium.CircleMarker(locations[0], color="green", tooltip=df["time (UTC)"].iloc[0], **kw).add_to(m)
    folium.CircleMarker(locations[-1], color="red", tooltip=df["time (UTC)"].iloc[-1], **kw).add_to(m)

In [ ]:
from folium.plugins import Fullscreen


m = folium.Map(tiles=tiles, attr=attr)
Fullscreen().add_to(m)

for k, row in df.iterrows():
    name = row["datasetID"]
    popup = glider_track_png(row)
    track = request_track(name)
    if track is not None:
        plot_track(track, popup)


m.fit_bounds(m.get_bounds())
m

In [ ]:
pathlib.Path("html/").mkdir(parents=True, exist_ok=True)

name = f"html/{today:%Y-%m-%d}-index.html"

m.save(name)

In [ ]:
import os
import tweepy


def twitter_authentication():
    auth = tweepy.OAuthHandler(os.environ["CONSUMER_KEY"], os.environ["CONSUMER_KEY_SECRET"])
    auth.set_access_token(os.environ["ACCESS_TOKEN"], os.environ["ACCESS_TOKEN_SECRET"])
    return tweepy.API(auth)


html_map_url = f"https://ocefpaf.github.io/glideroftheday/{today:%Y-%m-%d}-index.html"
png_map_file = f"images/{today:%Y-%m-%d}-map.png"
tweet = f"""There are {len(df["datasetID"])} gliders from {len(df["institution"].unique())} instituions in the water today.
Follow the link {html_map_url} for more information."""

In [ ]:
print(tweet)

In [ ]:
api = twitter_authentication()

media = api.media_upload(png_map_file)
post_result = api.update_status(status=tweet, media_ids=[media.media_id])